In [11]:
import re
import pandas as pd
from google.cloud import bigquery
import vertexai 
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part
import vertexai.preview.generative_models as generative_models 
import sys
import os

In [13]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\flori\\OneDrive\\Desktop\\LDP\\ldp-data-genai-producto-b871ebffc681.json"

In [2]:
##curso...pongo el id del proyecto del gcp de prueba q inicie con este mail
# Define project information
PROJECT_ID = "ldp-data-genai-producto"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
from vertexai.generative_models import Content, GenerativeModel, GenerationConfig

In [5]:
from google.cloud import bigquery

In [6]:
# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project='ldp-data-wyscout')

In [7]:
model = GenerativeModel("gemini-1.5-flash-001")

In [8]:
generation_config = GenerationConfig(
    temperature=0,
    top_p=0.0,
    top_k=1,
    candidate_count=1,
    max_output_tokens=8192,
)

In [19]:
safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

In [15]:
###ME TRAIGO LOS DATOS DE LA TABLA DE CAPOLOGY

# Consulta SQL
query2 = """SELECT * FROM `ldp-data-wyscout.capology.salarios`"""

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame
    df2 = client.query(query2).to_dataframe()
    print(df2.head())
except Exception as e:
    print(f"An error occurred: {e}")


   region            pais                      liga              club  \
0  Europa         Ucrania  Ukrainian Premier League  Inhulets Petrove   
1  Europa         Ucrania  Ukrainian Premier League             Minaj   
2  Europa         Ucrania  Ukrainian Premier League         Rukh Lviv   
3    Asia  Arabia Saudita          Saudi Pro League           Al Ahli   
4    Asia  Arabia Saudita          Saudi Pro League        Al Ettifaq   

   weeklyavg  annualavg moneda  
0       <NA>       <NA>         
1       <NA>       <NA>         
2       <NA>       <NA>         
3     106842    5555806  euros  
4      40740    2118462  euros  


In [ ]:
df2.head(20)

In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   region     570 non-null    object
 1   pais       570 non-null    object
 2   liga       570 non-null    object
 3   club       570 non-null    object
 4   weeklyavg  566 non-null    Int64 
 5   annualavg  566 non-null    Int64 
 6   moneda     570 non-null    object
dtypes: Int64(2), object(5)
memory usage: 32.4+ KB


In [21]:
dftext1 = df2.to_csv(index=False)

In [22]:
##salida armoa
def generate():
  vertexai.init(project=PROJECT_ID, location=LOCATION)
  responses = model.generate_content(
      [dftext1, """Podrias decirme que informacion ves en este csv?"""],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
  )

  for response in responses:
    print(response.text, end="")


generate()

El CSV contiene información sobre los clubes de fútbol de diferentes ligas en todo el mundo. Cada fila representa un club y contiene la siguiente información:

* **region:** La región del mundo donde se encuentra el club (Europa, Asia, América del Norte, América del Sur).
* **pais:** El país donde se encuentra el club.
* **liga:** La liga de fútbol en la que juega el club.
* **club:** El nombre del club.
* **weeklyavg:** El promedio semanal de ingresos del club (en euros o libras, dependiendo de la moneda).
* **annualavg:** El promedio anual de ingresos del club (en euros o libras, dependiendo de la moneda).
* **moneda:** La moneda en la que se expresan los ingresos del club (euros o libras).

Por ejemplo, la primera fila del CSV indica que el club Inhulets Petrove juega en la Ukrainian Premier League en Ucrania, y no se dispone de información sobre sus ingresos semanales o anuales.

El CSV proporciona una visión general de los ingresos de los clubes de fútbol en diferentes ligas del m

In [23]:
query22 = """SELECT * FROM `ldp-data-wyscout.capology.salarios_copy`"""

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame
    df22 = client.query(query22).to_dataframe()
    print(df22.head())
except Exception as e:
    print(f"An error occurred: {e}")


          region           pais liga                  club  weeklyavg  \
0  Norte America  United States  MLS  San Jose Earthquakes       7804   
1  Norte America  United States  MLS          Orlando City       8178   
2  Norte America         Canada  MLS   Vancouver Whitecaps       9843   
3  Norte America  United States  MLS          Charlotte FC       6711   
4  Norte America  United States  MLS        Houston Dynamo       9720   

   annualavg   moneda  
0     405828  dolares  
1     425248  dolares  
2     511836  dolares  
3     348971  dolares  
4     505434  dolares  


In [24]:
dftext22 = df22.to_csv(index=False)

In [67]:

# Consulta SQL
query = """SELECT wyId, officialName FROM ldp-data-wyscout.wyscout_curated.dict_teams
       where team_type="parent" and category="default" ; """

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame                     ###domestic_competition_name
    df = client.query(query).to_dataframe()
    print(df.head())
except Exception as e:
    print(f"An error occurred: {e}")


   wyId              officialName
0  1813  North Ferriby United AFC
1  1819          Witton Albion FC
2  1812           Matlock Town FC
3  1822   East Thurrock United FC
4  1810         Lincoln United FC


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34753 entries, 0 to 34752
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   wyId          34753 non-null  Int64 
 1   officialName  34753 non-null  object
dtypes: Int64(1), object(1)
memory usage: 577.1+ KB


In [69]:
df.head()

,wyId,officialName
0,1813,North Ferriby United AFC
1,1819,Witton Albion FC
2,1812,Matlock Town FC
3,1822,East Thurrock United FC
4,1810,Lincoln United FC


In [80]:
dftext = df.to_csv(index=False)

In [82]:
prompt = f"""Sos un agente que analiza datos necesito que generes una tabla con 2 columnas, a partir del analisis de 2 tablas.
La primera tabla contiene una columna con el nombre de 570 clubes y la segunda tabla contiene 2 columnas "wyId","officialName" con  34753 valores. Lo que yo necesito que hagas es, encontrar las coincidencias
entre los valores de la columna "club" de la tabla:{dftext1} con los valores de la columna "officialName" de la tabla:{dftext} , y a partir de esas coincidencias obtener el valor del id en el campo "wyId" de la tabla:{dftext}.
Es decir solamente tienen que quedar en el resultado, los valores que coinciden dentro de las dos tablas con su ID, deberian ser la misma cantidad de valores que la tabla : {dftext1}
La primera columna se conforma con el valor "wyId" de la tabla:{dftext}, con los valores que quedan luego de la coincidencia entre las otras columnas.
La segunda columna se conforma de la siguiente manera: unir el contenido de la la columna "club" de la tabla:{dftext} con la columna "officialName" de la tabla:{dftext1} buscando coincidencias y generando un valor unico final.


   """
# Generar la respuesta utilizando el prompt actualizado
responses = model.generate_content(prompt, generation_config=generation_config, stream=True)

# Inicializar una variable para almacenar el resultado completo
full_response = ""

# Recopilar y unir todas las partes de la respuesta
for response in responses:
    full_response += response.text

# Imprimir o usar la respuesta completa almacenada en la variable
print(full_response)

InvalidArgument: 400 Unable to submit request because the input token count is 1652366 but model only supports up to 1000000. Reduce the input token count and try again. You can also use the CountTokens API to calculate prompt token count and billable characters. Learn more: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models

In [73]:
matches = re.findall(r'\|\s*(.*?)\s*\|\s*(.*?)\s*\|', full_response)

# Convierte los datos en un DataFrame de pandas y asigna las columnas correctamente
df3 = pd.DataFrame(matches)

df3 = df3[~df3.eq('---').all(axis=1)]
df3.columns = df.iloc[0]  # La primera fila se convierte en nombres de columna
df3 = df[1:]
# Resetea el índice
df3.reset_index(drop=True, inplace=True)

print(df3)

        wyId                              officialName
0       1819                          Witton Albion FC
1       1812                           Matlock Town FC
2       1822                   East Thurrock United FC
3       1810                         Lincoln United FC
4       1823                                Fisher  FC
...      ...                                       ...
34747  67851  North Greenville Trailblazers University
34748  67850               Anderson Trojans University
34749  71728               Associação Atlética Maguary
34750  71739                   Lexington Sporting Club
34751  71744                          United IK Nordic

[34752 rows x 2 columns]


In [74]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34752 entries, 0 to 34751
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   wyId          34752 non-null  Int64 
 1   officialName  34752 non-null  object
dtypes: Int64(1), object(1)
memory usage: 577.1+ KB


In [75]:
df3.head(40)

,wyId,officialName
0,1819,Witton Albion FC
1,1812,Matlock Town FC
2,1822,East Thurrock United FC
3,1810,Lincoln United FC
4,1823,Fisher FC
5,1807,Guiseley AFC
6,1817,Wakefield FC
7,1808,Ilkeston FC
8,1818,Whitby Town FC
9,1814,Ossett Town FC


In [76]:
df2.head(40)

,liga,club
0,Swiss Super League,Basel
1,Swiss Super League,Grasshopper
2,Swiss Super League,Lausanne
3,Swiss Super League,Lugano
4,Swiss Super League,Luzern
5,Swiss Super League,Servette
6,Swiss Super League,Sion
7,Swiss Super League,St. Gallen
8,Swiss Super League,Winterthur
9,Swiss Super League,Young Boys


In [56]:
df.head(40)

,wyId,officialName,domestic_competition_name
0,4215,St Ouen-l'Aumône AS,None
1,4212,Association Sportive Excelsior,None
2,4218,Avant Garde de Plouvorn,None
3,4216,Entente Sportive Bressane Marboz,None
4,4214,FC Pyramid Grande Motte Littoral,None
5,4217,FC Annonay,None
6,4213,FC Versailles 78,National 1
7,4200,U.S. Saint-Malo,National 2
8,4204,US Blanzinoise,None
9,4205,ASPTT Mulhouse,None


In [57]:
df["officialName"].unique()

array(["St Ouen-l'Aumône AS", 'Association Sportive Excelsior',
       'Avant Garde de Plouvorn', ..., 'Tijuana Xolos USA',
       'Fresno Freeze FC', 'LA Villa FC'], dtype=object)

In [60]:
como=df["officialName"]=="Como"

In [63]:
values_to_check = ['Como']
mask = df['officialName'].isin(values_to_check)
print(df[mask])

Empty DataFrame
Columns: [wyId, officialName, domestic_competition_name]
Index: []
